In [ ]:
# to use this file, follow installation instructions at https://github.com/UKPLab/sentence-transformers

In [ ]:
# from transformers import AutoTokenizer, AutoModel

# tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# model = AutoModel.from_pretrained("bert-base-uncased")
# # # XLNet has no size limit (we are still limited by RAM)
# # # XLNet returns embedding for each token, unpooled
# # tokenizer = AutoTokenizer.from_pretrained("xlnet-base-cased")
# # model = AutoModel.from_pretrained("xlnet-base-cased")

# chunk = next(chunks(text, chunk_size=100))
# inputs = tokenizer(chunk, return_tensors="pt")
# outputs = model(**inputs)

# outputs.pooler_output[0]
# # outputs.last_hidden_state.shape

In [ ]:
try:
    %load_ext lab_black
except ModuleNotFoundError:
    print("nb_black not installed")

In [ ]:
import json
import pickle

from tqdm import tqdm
import pickledb
import numpy as np

from sentence_transformers import SentenceTransformer

In [ ]:
source = "Lightbulb-top-1000-results"
# source = "SomebodyMakeThis-top-1000-results"

In [ ]:
with open(f"../data/{source}.json") as file:
    data = json.load(file)
data = data["data"]

In [ ]:
url_to_text = {post["url"]: post["title"] + " " + post["selftext"] for post in data}

In [ ]:
def chunks(text, chunk_size=100, stride=0.8):
    words = text.split(" ")
    displacement = int(chunk_size * stride)

    start = 0
    while start < len(words):
        chunk = words[start : start + chunk_size]
        yield " ".join(chunk)
        start += displacement


def chunk_and_embed(text, model):
    # TODO now each chunk has a constant number of words, but maybe it should have a constant number of tokens
    embeddings = []
    for chunk in chunks(text, chunk_size=80):
        embedding = model.encode(chunk)
        embeddings.append(embedding)

    final_embedding = np.average(embeddings, axis=0)
    return final_embedding

In [ ]:
model = SentenceTransformer("paraphrase-MiniLM-L6-v2")

In [ ]:
url_to_embedding = dict()
for url in tqdm(url_to_text.keys(), ncols=80, smoothing=0.05):
    text = url_to_text[url]
    embedding = chunk_and_embed(text, model)
    url_to_embedding[url] = embedding

In [ ]:
with open(f"../data/{source}-embeddings.pickle", "wb") as file:
    pickle.dump(url_to_embedding, file)